# Simpsons characters dialogs

In [73]:
import pandas as pd
from flask import Flask, request

In [74]:
df = pd.read_csv('input/simpsons_dataset.csv', engine = 'python')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [75]:
nombres = { 
    'raw_character_text':'Characters',
    'spoken_works':'Text'
}
df = df.rename(columns=nombres)

# Create characters groups

In [76]:
df = df = df.dropna()
df = df[(df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson', 'Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince', 'Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson')))]
df.head()

,Characters,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
5,Martin Prince,I don't think there's anything left to say.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!


In [77]:
df['Groups'] = df['Groups'][df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson'))] = 'Home'
df['Groups'][df['Characters'].isin(('Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince'))] = 'School'
df['Groups'][df['Characters'].isin(('Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson'))] = 'Bar'

In [78]:
df_reduced = df.head(100)
df_reduced.head()

,Characters,spoken_words,Groups
1,Lisa Simpson,Where's Mr. Bergstrom?,Home
3,Lisa Simpson,That life is worth living.,Home
5,Martin Prince,I don't think there's anything left to say.,School
7,Bart Simpson,Victory party under the slide!,Home
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!,Home


# Including Simpsons characters on MongoDB

In [69]:
import requests

In [98]:
ch = df[['Characters', 'Groups']].drop_duplicates()
ch

,Characters,Groups
1,Lisa Simpson,Home
5,Martin Prince,School
7,Bart Simpson,Home
18,Nelson Muntz,School
22,Milhouse Van Houten,School
57,Homer Simpson,Home
60,Marge Simpson,Home
128,Moe Szyslak,Bar
139,Barney Gumble,Bar
473,Jimbo Jones,School


In [70]:
for character in pd.unique(df_reduced['Characters']):
    data = requests.get(f"http://localhost:8800/user/create/{character}").content
print(data)

b'\n    <img src="https://cdn.memegenerator.es/imagenes/memes/full/4/19/4199711.jpg">\n    '


# Creating chats

In [104]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/create/{group}").content
print(data)

b'{\n  "Group": "Bar"\n}\n'


# Adding characters to chat

In [103]:
for index, row in ch.iterrows():
    print(row['Characters'], row['Groups'])
    users = requests.get(f"http://localhost:8800/user/create/{row['Characters']}/{row['Groups']}").content
    print(users)

Lisa Simpson Home
b'{\n  "user_id": "5e51b56d5fa34008598fc6af"\n}\n'
Martin Prince School
b'{\n  "user_id": "5e51b56d5fa34008598fc6b0"\n}\n'
Bart Simpson Home
b'{\n  "user_id": "5e51b56d5fa34008598fc6b1"\n}\n'
Nelson Muntz School
b'{\n  "user_id": "5e51b56d5fa34008598fc6b2"\n}\n'
Milhouse Van Houten School
b'{\n  "user_id": "5e51b56d5fa34008598fc6b3"\n}\n'
Homer Simpson Home
b'{\n  "user_id": "5e51b56d5fa34008598fc6b4"\n}\n'
Marge Simpson Home
b'{\n  "user_id": "5e51b56d5fa34008598fc6b5"\n}\n'
Moe Szyslak Bar
b'{\n  "user_id": "5e51b56d5fa34008598fc6b6"\n}\n'
Barney Gumble Bar
b'{\n  "user_id": "5e51b56d5fa34008598fc6b7"\n}\n'
Jimbo Jones School
b'{\n  "user_id": "5e51b56d5fa34008598fc6b8"\n}\n'
Carl Carlson Bar
b'{\n  "user_id": "5e51b56d5fa34008598fc6b9"\n}\n'
Lenny Leonard Bar
b'{\n  "user_id": "5e51b56d5fa34008598fc6ba"\n}\n'
Ralph Wiggum School
b'{\n  "user_id": "5e51b56d5fa34008598fc6bb"\n}\n'
Seymour Skinner School
b'{\n  "user_id": "5e51b56d5fa34008598fc6bc"\n}\n'
Groundskeeper

In [124]:

for character in pd.unique(df_reduced['Characters']):
    print(character)
    usersdata = requests.get(f"http://localhost:8800/chat/adduser/{character}").text
    print(usersdata)

Lisa Simpson
User added to the chat
Martin Prince
User added to the chat
Bart Simpson
User added to the chat
Nelson Muntz
User added to the chat
Milhouse Van Houten
User added to the chat
Homer Simpson
User added to the chat
Marge Simpson
User added to the chat
Moe Szyslak
User added to the chat


# Adding messages to chat

In [125]:
'''
Characters from the same group will be on the same MongoDB document conversation
'''
for index, row in df_reduced.iterrows():  
    data = requests.get(f"http://localhost:8800/chat/{row['Groups']}/{row['Characters']}/{row['spoken_words']}").content
print(data)

b'Great. Included'


# Pruebas

In [30]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps

In [31]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()


In [137]:
for index, row in df_reduced.iterrows():
    i = row['Groups']
    print(i)
    query = list(db['Conversations'].find({'Group':i}))[0]['Message']
    print(type(query))

Home
<class 'list'>
Home
<class 'list'>
School
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
School
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
School
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
School
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 'list'>
Home
<class 